In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [3]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier

In [4]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score

In [36]:
from sklearn.ensemble import RandomForestRegressor

In [131]:
from sklearn import model_selection

In [13]:
df_predict=pd.read_csv('train_crystals.csv',index_col='identifiers')

In [14]:
df_predict.head()

,n_heavy_atoms,smiles,a,b,c,alpha,beta,gamma,z_value,z_prime,...,cell_volume,calculated_density,packing_coefficient,void_volume,is_centrosymmetric,is_sohncke,vdw_inter,vdw_intra,hbond_inter,hbond_intra
identifiers,,,,,,,,,,,,,,,,,,,,,
COXSUE,12,CC(=O)NN=C(C)C1=NCCS1,4.7362,17.0760,11.4150,90.0,94.432,90.0,4.0,1.0,...,920.431558,1.336802,0.667769,0.0,True,False,8,6,1,0
TUWFAU,15,CC(C)N(C)c1ccc(cc1C#N)C#N,8.9832,12.6212,9.6863,90.0,92.414,90.0,4.0,1.0,...,1097.246124,1.206163,0.689272,0.0,True,False,4,16,0,0
DUTHEG,14,CC1=CC(=O)N2C(=NC(=NC2=N1)C)N,11.1369,18.9130,4.0311,90.0,90.000,90.0,4.0,1.0,...,849.079420,1.495633,0.729064,0.0,False,False,7,4,1,1
AXUDED08,10,COC(=O)c1ccccc1,13.1840,5.9231,16.9510,90.0,90.000,90.0,8.0,1.0,...,1323.706139,1.366336,0.728081,0.0,True,False,4,4,0,0
ARAGUV,10,CC(=O)NC1=NNC(=C1)C,18.6000,19.9320,7.7992,90.0,90.000,90.0,16.0,1.0,...,2891.437972,1.278655,0.660441,0.0,False,False,5,2,2,0


In [86]:
d_1=df_predict['calculated_density']
d_1.head()

identifiers
COXSUE      1.336802
TUWFAU      1.206163
DUTHEG      1.495633
AXUDED08    1.366336
ARAGUV      1.278655
Name: calculated_density, dtype: float64

In [51]:
df_descriptors=pd.read_csv('train_descriptors.csv',index_col='identifiers')

In [64]:
df_descriptors.shape

(13449, 1617)

In [65]:
df_descriptors.isna().sum().sort_values(ascending=False)

MAXsssSnH     13449
MAXdssSe      13449
MINssBe       13449
MINsLi        13449
MAXssssPb     13449
              ...  
NsssGeH           0
NssssGe           0
NsAsH2            0
NssAsH            0
Unnamed: 0        0
Length: 1617, dtype: int64

In [66]:
df_descriptors.head()

,Unnamed: 0,name,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,InchiKey,SMILES
identifiers,,,,,,,,,,,,,,,,,,,,,
COXSUE,11365,Optimised_Molecules_V3.3460,8.773011,8.632564,0,0,14.599164,2.244739,4.388162,14.599164,...,185.062283,8.046186,219,11,54.0,58.0,4.833333,2.777778,UYSCRWBEPUAZRA-UHFFFAOYSA-N,CC(=O)NN=C(C)C1=NCCS1
TUWFAU,58476,Optimised_Molecules_V3.19496,10.813451,10.346393,0,0,18.488707,2.357869,4.715737,18.488707,...,199.110947,7.111105,364,22,70.0,80.0,6.805556,3.583333,RYPSYCXCJSHDJX-UHFFFAOYSA-N,CC(C)N(C)c1ccc(C#N)cc1C#N
DUTHEG,31879,Optimised_Molecules_V3.5074,10.922841,9.584172,0,0,16.685332,2.436274,4.872549,16.685332,...,191.080710,8.307857,271,22,74.0,87.0,5.666667,3.000000,PVRUUABKIQNQFL-UHFFFAOYSA-N,Cc1cc(=O)n2c(N)nc(C)nc2n1
AXUDED08,2885,Optimised_Molecules_V3.988,7.140017,6.873356,0,0,12.857279,2.214320,4.428639,12.857279,...,136.052429,7.558468,121,11,44.0,48.0,3.722222,2.444444,QPJVMBTYPHYUOC-UHFFFAOYSA-N,COC(=O)c1ccccc1
ARAGUV,2660,Optimised_Molecules_V3.795,7.399237,7.308033,0,0,11.638727,2.249778,4.311276,11.638727,...,139.074562,7.319714,125,8,46.0,49.0,4.333333,2.250000,LUFRABHJXNJTNZ-UHFFFAOYSA-N,CC(=O)Nc1cc(C)[nH]n1


In [67]:
df_descriptors.dropna(inplace=True, axis=1)

In [68]:
df_descriptors.shape

(13449, 988)

In [85]:
for i in df_descriptors.columns:
    print(i)

Unnamed: 0
name
ABC
ABCGG
nAcid
nBase
SpAbs_A
SpMax_A
SpDiam_A
SpAD_A
SpMAD_A
LogEE_A
VE1_A
VE2_A
VE3_A
VR1_A
VR2_A
nAromAtom
nAromBond
nAtom
nHeavyAtom
nSpiro
nBridgehead
nHetero
nH
nB
nC
nN
nO
nS
nP
nF
nCl
nBr
nI
nX
ATS0dv
ATS1dv
ATS2dv
ATS3dv
ATS4dv
ATS5dv
ATS6dv
ATS7dv
ATS8dv
ATS0d
ATS1d
ATS2d
ATS3d
ATS4d
ATS5d
ATS6d
ATS7d
ATS8d
ATS0Z
ATS1Z
ATS2Z
ATS3Z
ATS4Z
ATS5Z
ATS6Z
ATS7Z
ATS8Z
ATS0m
ATS1m
ATS2m
ATS3m
ATS4m
ATS5m
ATS6m
ATS7m
ATS8m
ATS0v
ATS1v
ATS2v
ATS3v
ATS4v
ATS5v
ATS6v
ATS7v
ATS8v
ATS0se
ATS1se
ATS2se
ATS3se
ATS4se
ATS5se
ATS6se
ATS7se
ATS8se
ATS0pe
ATS1pe
ATS2pe
ATS3pe
ATS4pe
ATS5pe
ATS6pe
ATS7pe
ATS8pe
ATS0are
ATS1are
ATS2are
ATS3are
ATS4are
ATS5are
ATS6are
ATS7are
ATS8are
ATS0p
ATS1p
ATS2p
ATS3p
ATS4p
ATS5p
ATS6p
ATS7p
ATS8p
ATS0i
ATS1i
ATS2i
ATS3i
ATS4i
ATS5i
ATS6i
ATS7i
ATS8i
AATS0dv
AATS1dv
AATS0d
AATS1d
AATS0Z
AATS1Z
AATS0m
AATS1m
AATS0v
AATS1v
AATS0se
AATS1se
AATS0pe
AATS1pe
AATS0are
AATS1are
AATS0p
AATS1p
AATS0i
AATS1i
ATSC0dv
ATSC1dv
ATSC2dv
ATSC3dv
ATSC4dv
ATSC5dv

In [70]:
data=df_descriptors[['nAtom','nHetero',
'nH',
'nB',
'nC',
'nN',
'nO',
'nS',
'nP',
'nF',
'nCl',
'nBr',
'nI']]

In [76]:
d=pd.concat([data, d_1],axis=1)
d.head()

,nAtom,nHetero,nH,nB,nC,nN,nO,nS,nP,nF,nCl,nBr,nI,calculated_density
identifiers,,,,,,,,,,,,,,
COXSUE,23,5,11,0,7,3,1,1,0,0,0,0,0,1.336802
TUWFAU,28,3,13,0,12,3,0,0,0,0,0,0,0,1.206163
DUTHEG,23,6,9,0,8,5,1,0,0,0,0,0,0,1.495633
AXUDED08,18,2,8,0,8,0,2,0,0,0,0,0,0,1.366336
ARAGUV,19,4,9,0,6,3,1,0,0,0,0,0,0,1.278655


In [121]:
d.shape

(13449, 14)

In [75]:
X_train=d.drop('calculated_density',axis=1)
y_train=d['calculated_density']

In [122]:
rfr=RandomForestRegressor(n_estimators=500)
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [123]:
test_desc=pd.read_csv('test_descriptors.csv',index_col='identifiers')

In [124]:
X_test=test_desc[['nAtom','nHetero',
'nH',
'nB',
'nC',
'nN',
'nO',
'nS',
'nP',
'nF',
'nCl',
'nBr',
'nI']]

In [125]:
rfr_pred=rfr.predict(X_test)

In [129]:
rfr_pred

array([1.49514779, 1.56010094, 1.73196664, ..., 1.2444072 , 1.61353011,
       2.06129592])

In [132]:
cv_scores=model_selection.cross_val_score(rfr, X_train, y_train,scoring='neg_mean_absolute_error')

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [133]:
print('Average negative MAE:', round(np.mean(cv_scores),3))
print('Standard Deviation:', round(np.std(cv_scores),3))

Average negative MAE: -0.056
Standard Deviation: 0.001


In [134]:
np.savetxt('task_1_pred.csv',rfr_pred)